## Shadie demo (v.0.1.1)

In [1]:
import numpy as np
import shadie
import toytree

print("shadie", shadie.__version__)
print("toytree", toytree.__version__)

shadie 0.1.1
toytree 2.1.1-dev


### Create a simple chromosome

In [2]:
chrom = shadie.chromosome.explicit({
    (0, 1_200_000): shadie.NONCDS,
    (1_200_001, 1_400_000): shadie.EXON,
    (1_400_001, 1_600_000): shadie.INTRON,
    (1_600_001, 1_800_000): shadie.EXON,
    (1_800_001, 5_000_000): shadie.NONCDS,
})
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t535813942bd1452392b1414fba86426e"> name: noncds
interval: (0, 1200000)
ElementType: g3
coding: False name: exon
interval: (1200001, 1400000)
ElementType: g1
coding: True name: intron
interval: (1400001, 1600000)
ElementType: g2
coding: True name: exon
interval: (1600001, 1800000)
ElementType: g1
coding: True name: noncds
interval: (1800001, 5000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000

### Setup models

In [3]:
with shadie.Model() as model_1:
    model_1.initialize(
        chromosome=chrom, 
        generations=5000,
        mut=1e-8, 
        recomb=1e-8,
        trees_file="/tmp/test-2.trees",
    )
    model_1.reproduction.bryophyte("dio", 5000, 5000)

In [4]:
with shadie.Model() as model_2:
    model_2.initialize(
        chromosome=chrom, 
        generations=5000,
        mut=1e-8, 
        recomb=1e-8,
        trees_file="/tmp/test-3.trees",
    )
    model_2.reproduction.bryophyte("dio", 5000, 5000)

### Run two populations in parallel

Each simulation model is run from a different random seed, and writes to a different .trees file path. I also provide the path to my updated `slim` binary as an argument.

In [5]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
with ProcessPoolExecutor(2) as pool:
    for model in [model_1, model_2]:
        kwargs = {"binary": "/usr/local/bin/slim", "seed": np.random.randint(2**31)}
        pool.submit(model.run, **kwargs)

### Get post-processor

In [10]:
post = shadie.TwoSims(
    tree_files=["/tmp/test-2.trees", "/tmp/test-3.trees"],
    mut=1e-8,
    recomb=1e-8,
    popsize=5000,
    chromosome=chrom,
)

04:03 | INFO    | _report_ninds   | inds alive at time=0; simpop0=1029, simpop1=1029
04:03 | INFO    | _report_mutations | Keeping 52 existing mutations of type(s) {2, 3}.
04:03 | INFO    | _report_mutations | Keeping 714 existing mutations of type(s) {0, 2, 3}.


### Plot simulation summary

In [11]:
post.draw_tree_sequence(sample=6, seed=123);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1800.0px" height="425.0px" viewBox="0 0 1800.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1f178b79982b49cdbb9b9c3259dd2366"> name: noncds
interval: (0, 200000)
ElementType: g3
coding: False name: exon
interval: (200001, 400000)
ElementType: g1
coding: True name: intron
interval: (400001, 600000)
ElementType: g2
coding: True name: exon
interval: (600001, 800000)
ElementType: g1
coding: True name: noncds
interval: (800001, 1000000)
ElementType: g3
coding: False 0 200000 400000 600000 800000 1000000 <polygon points="50.0,108.49385073285784 59.4896331738437,142.46925366428934 59.4896331738437,312.34626832144664 227.59170653907498,312.34626832144664 227.59170653907498,142.46925366428934 180.15707277538917,108.49385073285784 180.15707277538917,108.49385073285784 227.59170653907498,108.49385073285784 227.59170653907498,108.49385073285784 59.4896331738437,108.49385073285784 59.4896331738437,108.49385073285784 50.0,108.49385073285784" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 0
interval: (0 - 65343)
tmrca: 17662.02 <polygon points="180.15707277538917,108.49385073285784 227.59170653907498,142.46925366428934 227.59170653907498,312.34626832144664 395.6937799043062,312.34626832144664 395.6937799043062,142.46925366428934 626.2900948393017,108.49385073285784 626.2900948393017,108.49385073285784 395.6937799043062,108.49385073285784 395.6937799043062,108.49385073285784 227.59170653907498,108.49385073285784 227.59170653907498,108.49385073285784 180.15707277538917,108.49385073285784" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 1
interval: (65343 - 289316)
tmrca: 17662.02 idx: 2
interval: (289316 - 427544)
tmrca: 17662.02 idx: 3
interval: (427544 - 430704)
tmrca: 17662.02 idx: 4
interval: (430704 - 452476)
tmrca: 17662.02 <polygon points="951.2893754666447,108.49385073285784 900.0,142.46925366428934 900.0,312.34626832144664 1068.1020733652313,312.34626832144664 1068.1020733652313,142.46925366428934 1290.6841562232712,108.49385073285784 1290.6841562232712,108.49385073285784 1068.1020733652313,108.49385073285784 1068.1020733652313,108.49385073285784 900.0,108.49385073285784 900.0,108.49385073285784 951.2893754666447,108.49385073285784" style="fill:rgb(100%,85.1%,18.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 5
interval: (452476 - 622863)
tmrca: 17662.02 idx: 6
interval: (622863 - 632603)
tmrca: 17662.02 <polygon points="1310.0853145544204,108.49385073285784 1236.2041467304623,142.46925366428934 1236.2041467304623,312.34626832144664 1404.3062200956938,312.34626832144664 1404.3062200956938,142.46925366428934 1333.6336198717559,108.49385073285784 1333.6336198717559,108.49385073285784 1404.3062200956938,108.49385073285784 1404.3062200956938,108.49385073285784 1236.2041467304623,108.49385073285784 1236.2041467304623,108.49385073285784 1310.0853145544204,108.49385073285784" style="fill:rgb(70.2%,70.2%,70.2%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 7
interval: (632603 - 644425)
tmrca: 17662.02 <polygon points="1333.6336198717559,108.49385073285784 1404.3062200956938,142.46925366428934 1404.3062200956938,312.34626832144664 1572.408293460925,312.34626832144664 1572.408293460925,142.46925366428934 1585.4582647236266,108.49385073285784 1585.4582647236266,108.49385073285784 1572.408293460925,108.49385073285784 1572.408293460925,108.49385073285784 1404.3062200956938,108.49385073285784 1404.3062200956938,108.49385073

### Plot individual trees

In [12]:
post.draw_tree(idx=0, sample=[10, 10], seed=123);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1f1ebbe23d9648259df207fdc3583178"> p1-19 p1-13 p1-11 p1-10 p1-16 p1-12 p1-15 p1-18 p1-17 p1-14 p0-5 p0-1 p0-0 p0-8 p0-2 p0-9 p0-6 p0-4 p0-7 p0-3 id: 0
pos: 13231
time: 3560
mtype: 0 id: 1
pos: 14927
time: 14989
mtype: 0 id: 2
pos: 15281
time: 9187
mtype: 0 id: 3
pos: 17751
time: 9019
mtype: 0 id: 4
pos: 19020
time: 2948
mtype: 0 id: 5
pos: 21018
time: 17280
mtype: 0 id: 6
pos: 21096
time: 8053
mtype: 0 id: 7
pos: 23055
time: 10549
mtype: 0 id: 8
pos: 23249
time: 7170
mtype: 0 id: 9
pos: 23673
time: 17505
mtype: 0 id: 10
pos: 28069
time: 13956
mtype: 0 id: 11
pos: 31416
time: 2027
mtype: 0 id: 12
pos: 36533
time: 728
mtype: 0 id: 13
pos: 37963
time: 8543
mtype: 0 id: 14
pos: 43221
time: 8286
mtype: 0 id: 15
pos: 44932
time: 8503
mtype: 0 id: 16
pos: 45481
time: 12132
mtype: 0 id: 17
pos: 48943
time: 1539
mtype: 0 id: 18
pos: 50830
time: 4787
mtype: 0 id: 19
pos: 55479
time: 15290
mtype: 0 id: 20
pos: 56359
time: 15734
mtype: 0 id: 21
pos: 58077
time: 14587
mtype: 0 id: 22
pos: 59022
time: 9442
mtype: 0 id: 23
pos: 59803
time: 7121
mtype: 0 id: 24
pos: 60283
time: 6249
mtype: 0 id: 25
pos: 62511
time: 14801
mtype: 0 id: 26
pos: 62527
time: 10362
mtype: 0 id: 27
pos: 62528
time: 129
mtype: 0 15000 10000 5000 0 tree in interval 0 (sites 0-65343)

### Calculate statistics

In [13]:
post.stats(sample=10, reps=20)

,mean,CI_5%,CI_95%
theta_0,0.000005,0.000004,0.000005
theta_1,0.000004,0.000004,0.000004
Fst_01,0.978016,0.976729,0.979304
Dist_01,0.000389,0.000389,0.000389
D_Taj_0,-0.999130,-1.201882,-0.796378
D_Taj_1,-1.297371,-1.488726,-1.106015
